Tests et Profilage, Python
==========================

The purpose of this tutorial is to clarify certain concepts related to testing in
programming and to emphasize good insurance practices
quality in Python. We will also investigate how to profile its code.
to determine performance.

Unit tests
----------
The main pillar of quality assurance in programming
is unit tests. The purpose of unit tests is to verify that
my code does what it's supposed to do.

The unit tests consist of 4 steps:
1. Preparation (`setUp`): the components needed for the test are
   instantiated.
2. Exercise: The unit of code being tested is launched.
3. Verification: The output of the unit of code in question is compared to the output of its
   anticipated value.
4. TearDown: The components needed for the test are
   destroyed.

Un test unitaire est généralement conçu afin d'isoler une unité de code. Il
est important de découpler les tests unitaires.   Nous allons commencer par
nous habituer à la structure générale des tests. Le premier exemple est le
script `utilities.py` et sa classe de tests dans `test_utilities.py`.

In [ ]:
class TestUtilities:
    """
    This class is a classic example of xUnit test cases.
    The setup_class/teardown_class methods are called automatically at the beginning and end of unit tests belonging to this class.
    """
    @classmethod
    def setup_class(cls):
        cls.converter = StringConverters()

    @classmethod
    def teardown_class(cls):
        pass


    def test_translate_to_jadensmith_raises_value_error(self):
        with pytest.raises(ValueError):
            self.converter.translate_to_jadensmith(999)

    ....

We get our usual test structure with a test class. Note
that the class starts with the keyword `Test` which allows the pytest module to
include it. In addition, the tests in the class all start with the `test_` prefix.
for the same reason.

Pytest
------

Pytest allows us to run our tests and see the results.
with a graphical summary on the command line. The command is `pytest <your_test_file.py> `.

There are several useful options for getting started
such as `-s` which displays the _standard out_ (the print statements, or else they're not printed since pytest captures all the outbounds tests.

Pytest is one of the main test modules in Python. It supports
classic units as we've just seen. In addition, it allows to
handle tests differently with _fixture_. We will refactor the
class TestUtilities to use the _fixture_.
The file is *test_utilities_refactored.py*.

In [ ]:
@pytest.fixture(autouse=True, scope='module')
def create_converter():
    converter = StringConverters()
    yield converter

def test_translate_to_jadensmith_raises_value_error(create_converter):
    with pytest.raises(ValueError):
        create_converter.translate_to_jadensmith(999)

def test_translate_to_jadensmith_single_word(create_converter):
    sentence = "hello"
    expected_value = "Hello"

    actual_value = create_converter.translate_to_jadensmith(sentence)

    assert expected_value == actual_value


The _fixtures_ are a very powerful feature of pytest.

Pytest Coverage
---------------

_pytest-cov_ is an additional module allowing us to visualize the
proportion of the code tested. It is easy to narrow down to just a few possible paths through the code.
For example, if I have the following code:

In [ ]:
if first_name == "":
   if last_name == "":
        db.set_name("unknown")
   else:
        db.set_name(last_name)
else:
    if last_name == "":
        db.set_name(first_name + "_unknown")
    else:
        db.set_name(first_name +  " " + last_name)

We have to write either 4 tests or one test that goes all the way.
A module like _pytest-cov_ allows us to compute the proportion of the code which is actually tested.

The command is `pytest --cov=<module name> <test_directory>`

![pytest cov result](./include/pytest_cov.png)

The previous picture shows the result of a test coverage.

Benchmarking in Python
----------------------
Sometimes it is important to meet certain performance criteria.
How to validate the performance of specific components of our software?
For isolated functions, we can use the `timeit` module.

In [ ]:
import timeit
s = """
sc = StringConverters()
with open("sherlock.txt") as f:
    lines = f.readlines()
meme_lines = [sc.translate_to_internet_meme(l) for l in lines[:100]]
"""
print(timeit.timeit(stmt=s, setup="from __main__ import  StringConverters", number=1000))

The previous code opens a text file (Sherlock Holmes) and calls the `translate_to_internet_meme` function. 
on the first 100 sentences. This procedure is repeated 1000 times. This results in 
a fairly accurate performance average.   
Note that the code is in `str` form which limits the testing possibilities.

Profiling en Python
-------------------
In addition to benchmarking, profiling can also be useful in order to better understand one's program.
Profiling is a set of descriptive statistics on the execution time of the various components of a program.

The most popular and useful module is probably the `cprofile` module.
This module is designed to create a runtime profile, not for benchmarking purposes, use `timeit` in this context.

The command to run the *cprofile* profiler is as follows:
`python -m cprofile <your_script_name.py`

In [ ]:
import time

def f(x):
    x = str(x)
    c = "this value is " + x +  "."
    time.sleep(1)
    print(c)

l = [1,2,3,4,5,6,7]

for v in l:
    f(v)

![cprofile](./include/result_cprofile.png)

Here is the meaning of the columns:
* ncalls: the number of times the function has been called.
* tottime: the total time spent in a function.
* percall: tottime / ncalls
* cumtime: cumulative time spent in a function and its sub-functions.
